In [ ]:
#ahora modelado y analsis comparativo

In [16]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

#Luego, cuando filtramos las clases con ≥2 ejemplos, algunas etiquetas desaparecieron 
#XGBoost no tolera esos saltos.Hay que relabel las clases después de filtrar para que sean continuas (0, 1, 2, 3, 4, 5,...).
#volver a usar LabelEncoder después de filtrar.
from sklearn.preprocessing import LabelEncoder

# Variables
X = tfidf_dataframe
y = data['SubRubro_encoded']

#***********agrego esto porque daba error

# Contar cuántas veces aparece cada clase
conteo_clases = data['SubRubro_encoded'].value_counts()

# Filtrar para quedarnos solo con las clases que tienen al menos 2 productos
clases_validas = conteo_clases[conteo_clases >= 2].index

# Filtrar el dataset
data_filtrado = data[data['SubRubro_encoded'].isin(clases_validas)]

# *** NUEVO: Reasignar etiquetas consecutivas ***
le2 = LabelEncoder()
data_filtrado['SubRubro_encoded_relabel'] = le2.fit_transform(data_filtrado['SubRubro_encoded'])
# SubRubro_encoded_relabel es una nueva columna con las etiquetas consecutivas correctas.

# Redefinir X e y
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel']  # Ahora usamos la versión "relabel"
#**********************************************


# Definir todas las clases posibles (importante para XGBoost)
todas_las_clases = np.unique(y)



C:\Users\Usuario\AppData\Local\Temp\ipykernel_12952\3178245203.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtrado['SubRubro_encoded_relabel'] = le2.fit_transform(data_filtrado['SubRubro_encoded'])


In [62]:
# -------------------------------------
#     Aplicación de modelos
# -------------------------------------
# ⚠️ACLARACIONES:
# Aqui me dio error porque algunas clases Tienes clases en SubRubro_encoded que solo tienen 1 ejemplo.
# Cuando pides stratify=y, Sklearn necesita mínimo 2 muestras por clase para poder dividir entre train y test.
# No se puede dividir 1 solo dato en train/test, por eso explota.
# el dataset tiene algunos subrubros muy raros o casi vacíos (con 1 solo producto).
# Esto complica el modelado porque no se puede entrenar ni testear bien con una clase así.
# entonces debi eliminar el dataset aquellas clases (SubRubro_encoded) que tienen menos de 2 ejemplos
# -------------------------------------
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

#Luego, cuando filtramos las clases con ≥2 ejemplos, algunas etiquetas desaparecieron 
#XGBoost no tolera esos saltos.Hay que relabel las clases después de filtrar para que sean continuas (0, 1, 2, 3, 4, 5,...).
#volver a usar LabelEncoder después de filtrar.
from sklearn.preprocessing import LabelEncoder

# Variables
X = tfidf_dataframe
y = data['SubRubro_encoded']

#***********agrego esto porque daba error

# Contar cuántas veces aparece cada clase
conteo_clases = data['SubRubro_encoded'].value_counts()

# Filtrar para quedarnos solo con las clases que tienen al menos 2 productos
clases_validas = conteo_clases[conteo_clases >= 2].index

# Filtrar el dataset
data_filtrado = data[data['SubRubro_encoded'].isin(clases_validas)]

# *** NUEVO: Reasignar etiquetas consecutivas ***
le2 = LabelEncoder()
data_filtrado['SubRubro_encoded_relabel'] = le2.fit_transform(data_filtrado['SubRubro_encoded'])
# SubRubro_encoded_relabel es una nueva columna con las etiquetas consecutivas correctas.

# Redefinir X e y
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel']  # Ahora usamos la versión "relabel"
#**********************************************



# Definir todas las clases posibles (importante para XGBoost)
todas_las_clases = np.unique(y)





C:\Users\Usuario\AppData\Local\Temp\ipykernel_1408\2656807289.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtrado['SubRubro_encoded_relabel'] = le2.fit_transform(data_filtrado['SubRubro_encoded'])


In [ ]:
## NUEVOS MODELOS##

In [ ]:
#--------------------------------------------------
#   🔎🔎 ✔️VALIDACIÓN
#  Busca de los hipermarametros para Random Forest
#--------------------------------------------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import time

# Datos
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel']

# Definir la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Instanciar modelo base
rf = RandomForestClassifier(random_state=42)

# Instanciar GridSearchCV
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,  # validación cruzada de 5 folds
                           scoring='f1_weighted',
                           n_jobs=-1,  # paraleliza el proceso
                           verbose=2)

# Ejecutar búsqueda
inicio = time.time()
grid_search.fit(X, y)
fin = time.time()

# Resultados
print("\nMejores hiperparámetros:")
print(grid_search.best_params_)

print(f"\nMejor F1-Score (cv): {grid_search.best_score_:.4f}")
print(f"Tiempo total de búsqueda: {fin - inicio:.2f} segundos")


In [50]:
#------------------------------------------------------------
#   💻💥 ✔️VALIDACIÓN
#  RANDOM FOREST - CROSS VALIDACION CON hiperparametros optimos
#-------------------------------------------------------------
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score

# --- Datos ---
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel'].reset_index(drop=True)

# --- Modelo con mejores hiperparámetros ---
rf_model = RandomForestClassifier(
    bootstrap=True,
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,#era 100
    random_state=42
)

# --- Validación cruzada ---
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    inicio = time.time()
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    fin = time.time()

    # Matriz de confusión global (todas clases)
    cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))
    TP = np.diag(cm)
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    # Evitar divisiones por cero
    precision_clase = np.divide(TP, TP + FP, out=np.zeros_like(TP, dtype=float), where=(TP + FP) != 0)
    recall_clase = np.divide(TP, TP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + FN) != 0)
    accuracy_clase = np.divide(TP + TN, TP + TN + FP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + TN + FP + FN) != 0)

    # Promedios macro por pliegue
    acc = accuracy_clase.mean()
    prec = precision_clase.mean()
    rec = recall_clase.mean()
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    tiempo = fin - inicio

    resultados.append({
        'Accuracy': acc,
        'Precisión': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Tiempo (s)': tiempo
    })

# --- Mostrar resultados por pliegue ---
resultados_df = pd.DataFrame(resultados)
print("\nResultados por pliegue:")
print(resultados_df.round(4))

# --- Promedios ---
promedios = resultados_df.mean().to_frame().T
print("\nPromedios:")
print(promedios.round(4))


C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



Resultados por pliegue:
   Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
0    0.9945     0.5512  0.5059    0.5459     14.5207
1    0.9951     0.5794  0.5613    0.6171     14.7311
2    0.9945     0.5300  0.5103    0.5719     15.1886
3    0.9944     0.5451  0.5026    0.5625     15.0333
4    0.9953     0.6103  0.5970    0.6454     13.9368

Promedios:
   Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
0    0.9948     0.5632  0.5354    0.5885     14.6821


In [ ]:
#----------------------------------------
#   🔎 VALIDACIÓN
#  Busca de los hipermarametros para SVM
#----------------------------------------
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, f1_score
import time

# 1. Datos
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel']

# 2. Definir la grilla de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10],                     # Regularización
    'kernel': ['linear', 'rbf'],          # Tipos de kernel
    'gamma': ['scale', 'auto']            # Solo para kernels no lineales
}

# 3. Definir modelo base
modelo_svm = SVC()

# 4. Definir GridSearchCV con F1-score como métrica
grid = GridSearchCV(estimator=modelo_svm,
                    param_grid=param_grid,
                    scoring=make_scorer(f1_score, average='weighted'),
                    cv=5,
                    n_jobs=-1,
                    verbose=2)

# 5. Ajuste con control de tiempo
inicio = time.time()
grid.fit(X, y)
fin = time.time()

# 6. Resultados
print("\nMejores hiperparámetros:")
print(grid.best_params_)
print(f"\nMejor F1-Score (cv): {grid.best_score_:.4f}")
print(f"Tiempo total de búsqueda: {fin - inicio:.2f} segundos")

#Mejores hiperparámetros:
#{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

#Mejor F1-Score (cv): 0.6211
#Tiempo total de búsqueda: 2122.02 segundos


In [34]:
#------------------------------------------------------------------
#    💻💥 ✔️VALIDACIÓN VALIDACIÓN
#  Aplicación Cross Validation con mejores hiperparámetros para SVM
#------------------------------------------------------------------

import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Supón que tienes el DataFrame `data_filtrado` ya cargado
# Paso 1: TF-IDF y extracción de variables
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
X = tfidf_vectorizer.fit_transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel'].reset_index(drop=True)

# Paso 2: Definir modelo SVM con los mejores hiperparámetros
modelo = SVC(C=10, gamma='scale', kernel='linear')

# Paso 3: Validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = []

for i, (train_index, test_index) in enumerate(kf.split(X, y), start=1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    inicio = time.time()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    fin = time.time()

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    tiempo = fin - inicio

    resultados.append({
        'Pliegue': i,
        'Accuracy': acc,
        'Precisión': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Tiempo (s)': tiempo
    })

# Mostrar resultados
df_resultados = pd.DataFrame(resultados)
print("\nResultados por pliegue:")
print(df_resultados.round(4))

print("\nPromedios:")
print(df_resultados[['Accuracy', 'Precisión', 'Recall', 'F1-Score', 'Tiempo (s)']].mean().round(4))



C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



Resultados por pliegue:
   Pliegue  Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
0        1    0.6885     0.7086  0.5055    0.6764     29.0638
1        2    0.7154     0.7287  0.5941    0.7040     28.8397
2        3    0.6996     0.7311  0.5443    0.6936     29.0091
3        4    0.6829     0.7110  0.5278    0.6727     29.1496
4        5    0.7497     0.7603  0.6466    0.7387     30.1970

Promedios:
Accuracy       0.7072
Precisión      0.7279
Recall         0.5636
F1-Score       0.6971
Tiempo (s)    29.2518
dtype: float64


In [ ]:
#----------------------------------------
#   🔎🔎 ✔️ VALIDACIÓN
#  Busca de los hipermarametros para Regresión Logística
#----------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import time

# --- Definir el espacio de búsqueda de hiperparámetros ---
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],            # Fuerza de regularización
    'penalty': ['l2'],                       # Penalización (l1 requiere 'liblinear')
    'solver': ['lbfgs', 'saga'],             # Algoritmos que soportan 'l2' y multiclase
    'multi_class': ['multinomial'],          # Para clasificación multiclase
    'max_iter': [500, 1000]                  # Iteraciones máximas para convergencia
}

# --- Instanciar modelo base ---
modelo_base = LogisticRegression()

# --- Definir el GridSearch ---
grid = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    scoring=make_scorer(f1_score, average='weighted'),
    cv=5,
    n_jobs=-1,
    verbose=2
)

# --- Ejecutar búsqueda ---
inicio = time.time()
grid.fit(X_train, y_train)
fin = time.time()

# --- Mostrar resultados ---
print("Mejores hiperparámetros:")
print(grid.best_params_)
print(f"\nMejor F1-Score (cv): {round(grid.best_score_, 4)}")
print(f"Tiempo total de búsqueda: {round(fin - inicio, 2)} segundos")

# Mejores hiperparámetros:
# {'C': 100, 'max_iter': 1000, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'saga'}

# Mejor F1-Score (cv): 0.6274
# Tiempo total de búsqueda: 53892.58 segundos


In [ ]:
# #-------------------------------------------------------------------------
#   💥💻⚠️ VALIDACIÓN
#  REGRESIÓN LOGÍSTICA - Cross validacion con  los hipermarametros óptimos
# #-------------------------------------------------------------------------
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Datos
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel'].reset_index(drop=True)

# Modelo con mejores hiperparámetros
modelo = LogisticRegression(
    C=100,
    max_iter=1000,
    multi_class='multinomial',
    penalty='l2',
    solver='saga'
)

# Validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Pliegue {fold+1}")
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    inicio = time.time()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    fin = time.time()
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    tiempo = fin - inicio
    
    resultados.append({
        'Accuracy': acc,
        'Precisión': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Tiempo (s)': tiempo
    })

# Mostrar resultados
df_resultados = pd.DataFrame(resultados)
print("\nResultados por pliegue:")
print(df_resultados.round(4))

print("\nPromedios:")
print(df_resultados.mean().round(4))

# Pliegue 5

# Resultados por pliegue:
#    Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
# 0    0.7064     0.5793  0.5241    0.5339   6896.2770
# 1    0.7423     0.6337  0.5985    0.5907   6948.1079
# 2    0.7137     0.5951  0.5395    0.5486   6898.3654
# 3    0.7060     0.5888  0.5347    0.5399   6870.5371
# 4    0.7497     0.6183  0.6093    0.5970   6869.4316

# Promedios:
# Accuracy         0.7236
# Precisión        0.6031
# Recall           0.5612
# F1-Score         0.5620
# Tiempo (s)    6896.5438
# dtype: float64



In [ ]:
#----------------------------------------
#   🔎🔎 ✔️ VALIDACIÓN
#  Busca de los hipermarametros para Naive Bayes
#----------------------------------------
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, f1_score
import time

# --- Datos (usamos los mismos ya preparados) ---
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel']

# --- Modelo base ---
modelo = MultinomialNB()

# --- Definir la grilla de parámetros ---
param_grid = {
    'alpha': [0.1, 0.3, 0.5, 0.7, 1.0]
}

# --- Configurar GridSearchCV ---
grid_search_nb = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=make_scorer(f1_score, average='weighted'),
    cv=5,
    n_jobs=-1,
    verbose=2
)

# --- Ejecutar búsqueda ---
inicio = time.time()
grid_search_nb.fit(X, y)
fin = time.time()

# --- Resultados ---
print("Mejores hiperparámetros:")
print(grid_search_nb.best_params_)

print(f"\nMejor F1-Score (cv): {grid_search_nb.best_score_:.4f}")
print(f"Tiempo total de búsqueda: {fin - inicio:.2f} segundos")

#Mejores hiperparámetros:
#{'alpha': 0.1}

#Mejor F1-Score (cv): 0.6143
#Tiempo total de búsqueda: 11.79 segundos


In [ ]:
#   💥💻⚠️ VALIDACIÓN --------------------------------------------
#  NAIVE BAYES - Aplicación de los hiperparametros encontrados  
#   Cross Validation
#------------------------------------------------------------------
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer

# --- Paso 1: Vectorización TF-IDF ---
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
X = tfidf_vectorizer.fit_transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel'].reset_index(drop=True)

# --- Paso 2: Modelo con mejor hiperparámetro ---
modelo = MultinomialNB(alpha=0.1)

# --- Paso 3: Validación cruzada ---
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = []

for i, (train_index, test_index) in enumerate(kf.split(X, y), start=1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    inicio = time.time()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    fin = time.time()

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))
    support = cm.sum(axis=1)

    TP = np.diag(cm)
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    # Accuracy global
    
    acc = np.trace(cm) / np.sum(cm) 

    # Métricas por clase
    precision_clase = np.divide(TP, TP + FP, out=np.zeros_like(TP, dtype=float), where=(TP + FP) != 0)
    recall_clase = np.divide(TP, TP + FN, out=np.zeros_like(TP, dtype=float), where=(TP + FN) != 0)
    f1_clase = np.divide(2 * precision_clase * recall_clase, precision_clase + recall_clase,
                         out=np.zeros_like(precision_clase, dtype=float), where=(precision_clase + recall_clase) != 0)

    # Promedios ponderados
    total = support.sum()
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    # prec = np.sum(precision_clase * support) / total
    # rec = np.sum(recall_clase * support) / total
    f1 = np.sum(f1_clase * support) / total
    tiempo = fin - inicio

    resultados.append({
        'Accuracy': acc,
        'Precisión': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Tiempo (s)': tiempo
    })

    

# --- Mostrar resultados ---
resultados_df = pd.DataFrame(resultados)
print("\nResultados por pliegue:")
print(resultados_df.round(4))

print("\nPromedios:")
print(resultados_df.mean().to_frame().T.round(4))

# Resultados por pliegue:
#    Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
# 0    0.6782     0.5222  0.4289    0.6458      0.1042
# 1    0.7090     0.5199  0.4366    0.6768      0.0794
# 2    0.6881     0.5149  0.4095    0.6619      0.0939
# 3    0.6765     0.5131  0.4291    0.6449      0.0945
# 4    0.7227     0.5451  0.4689    0.6948      0.1098

# Promedios:
#    Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
# 0    0.6949      0.523  0.4346    0.6648      0.0964



In [48]:
#----------------------------------------
#   🔎 VALIDACIÓN
#  Busca de los hipermarametros para XGBoost - 
#---------------------------------------------
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
#from sklearn.utils.fixes import loguniform
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# --- Datos: TF-IDF y etiquetas ---
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel']

# --- Modelo base ---
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    objective='multi:softmax',
    num_class=len(np.unique(y)),
    verbosity=1
)

# --- Grid más reducido (36 combinaciones en lugar de 384 o más) ---
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [4, 6],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'gamma': [0],
    'reg_lambda': [1],
    'min_child_weight': [1]
}

# --- GridSearchCV con logs visibles ---
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,
    verbose=2,  # Muestra progreso
    n_jobs=-1   # Usa todos los núcleos disponibles
)

# --- Ajustar modelo ---
grid_search.fit(X, y)

# --- Resultados ---
print("\n✅ Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)
print(f"📈 Mejor F1-Score (cv): {grid_search.best_score_:.4f}")

#Mejores hiperparámetros encontrados:
#{'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 150, 'reg_lambda': 1, 'subsample': 0.8}
#📈 Mejor F1-Score (cv): 0.4491


Fitting 3 folds for each of 8 candidates, totalling 24 fits


C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Usuario\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [12:19:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Mejores hiperparámetros encontrados:
{'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 150, 'reg_lambda': 1, 'subsample': 0.8}
📈 Mejor F1-Score (cv): 0.4491


In [47]:
#----------------------------------------------------------
#      💥💻⚠️ VALIDACIÓN
#  XGBoost Aplicación de los hiperparametros encontrados 
# Con Cros Validation
#----------------------------------------------------------
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score
from xgboost import XGBClassifier

# Datos
X = tfidf_vectorizer.transform(data_filtrado['Nombre_Limpio']).toarray()
y = data_filtrado['SubRubro_encoded_relabel'].reset_index(drop=True)

# Modelo con mejores hiperparámetros
modelo_xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    objective='multi:softmax',
    num_class=len(np.unique(y)),
    colsample_bytree=0.8,
    gamma=0,
    learning_rate=0.2,
    max_depth=6,
    min_child_weight=1,
    n_estimators=150, 
    reg_lambda=1,
    subsample=0.8,
    verbosity=0
)

# Validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Pliegue {fold+1}")
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    inicio = time.time()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    fin = time.time()
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    tiempo = fin - inicio
    
    resultados.append({
         'Pliegue': fold + 1,
        'Accuracy': acc,
        'Precisión': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Tiempo (s)': tiempo
    })




# Resultados
df_resultados = pd.DataFrame(resultados)
print("\nResultados por pliegue:")
print(df_resultados.round(4))

print("\nPromedios:")



print(df_resultados.drop(columns='Pliegue').mean().to_frame().T.round(4))

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Pliegue 1
Pliegue 2
Pliegue 3
Pliegue 4
Pliegue 5

Resultados por pliegue:
   Pliegue  Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
0        1    0.5474     0.2831  0.2583    0.2586     56.3128
1        2    0.5808     0.3228  0.3002    0.2917     72.6567
2        3    0.5481     0.2983  0.2680    0.2721     75.6915
3        4    0.5456     0.3117  0.2836    0.2875     85.5765
4        5    0.5751     0.3313  0.3073    0.2981     89.7105

Promedios:
   Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
0    0.5594     0.3094  0.2835    0.2816     75.9896


In [ ]:
#-----------------------------------------
# ⏱️🔎🔎✔️ Busqueda de hiperparametros óptimos para knn
#-----------------------------------------
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import time

# Definir el modelo base
knn = KNeighborsClassifier()

# Definir la grilla de hiperparámetros a probar
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Usar F1-Score como métrica de evaluación
scorer = make_scorer(f1_score, average='weighted')

# Configurar la búsqueda con validación cruzada
grid_search = GridSearchCV(estimator=knn,
                           param_grid=param_grid,
                           scoring=scorer,
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

# Tiempo de ejecución
inicio = time.time()
grid_search.fit(X, y)
fin = time.time()

# Mostrar resultados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)
print(f"\n📈 Mejor F1-Score (cv): {grid_search.best_score_:.4f}")
print(f"⏱️ Tiempo total de búsqueda: {fin - inicio:.2f} segundos")

#Mejores hiperparámetros encontrados:
#{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}

#📈 Mejor F1-Score (cv): 0.5675
#⏱️ Tiempo total de búsqueda: 263.99 segundos



In [ ]:
#----------------------------------------------------------
#    💥💻 VALIDACIÓN
#  KNN Aplicación de los hiperparametros encontrados 
# Cross Validation
#----------------------------------------
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import time
import pandas as pd

# Mejor configuración hallada
mejores_params = {
    'n_neighbors': 3,
    'weights': 'distance',
    'metric': 'euclidean'
}

# Inicializar modelo con mejores parámetros
modelo_knn = KNeighborsClassifier(**mejores_params)

# Inicializar validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Almacenar resultados por pliegue
resultados_knn = []

for train_idx, test_idx in kf.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    inicio = time.time()
    modelo_knn.fit(X_train, y_train)
    y_pred = modelo_knn.predict(X_test)
    fin = time.time()

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    tiempo = fin - inicio

    resultados_knn.append({
        'Accuracy': acc,
        'Precisión': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Tiempo (s)': tiempo
    })

# Mostrar resultados
df_resultados_knn = pd.DataFrame(resultados_knn)
print(df_resultados_knn.round(4))
promedios = df_resultados_knn.mean().to_frame().T
promedios.index = ['Promedio']
print(promedios.round(4))

#Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
#0    0.5808     0.7656  0.5808    0.6469      0.2812
#1    0.6026     0.7742  0.6026    0.6631      0.2846
#2    0.5892     0.7701  0.5892    0.6533      0.3040
#3    0.6021     0.7786  0.6021    0.6581      0.2986
#4    0.6367     0.8099  0.6367    0.6980      0.2644
 #         Accuracy  Precisión  Recall  F1-Score  Tiempo (s)
#Promedio    0.6023     0.7797  0.6023    0.6639      0.2865


In [ ]:
#  fin modelos validacion 💯 🔚-----------------------------------------